# Mount drive

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Install dependencies

In [2]:
!pip install lazypredict

# Import libraries

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import gensim.downloader
from lazypredict.Supervised import LazyClassifier

# SEED for reproducibility
import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Load data

In [30]:
X_column = 'RequirementText'
y_column = 'Class'

CLASSICAL_PATH = '/content/drive/MyDrive/Tesi/Master-Thesis/data/processed/quantum.csv'

df = pd.read_csv(CLASSICAL_PATH, index_col=False)

# Vectorize train set

In [31]:
def getTrainSet(df, vectorization_technique='BoW'):
    if vectorization_technique == 'BoW':
        countvec = CountVectorizer(max_features=100)
        vectors = countvec.fit_transform(df[X_column]).toarray()
    elif vectorization_technique == 'TFIDF':
        countvec = CountVectorizer(max_features=100)
        bow = countvec.fit_transform(df[X_column]).toarray()
        tfidfconverter = TfidfTransformer()
        vectors = tfidfconverter.fit_transform(bow).toarray()
    elif vectorization_technique == 'Word2Vec':
        w2v_pretrained = gensim.downloader.load('word2vec-google-news-300')
        vectors = []
        for msg in df[X_column]:
            words = msg.split()
            vecs = [w2v_pretrained[word][:300] for word in words if word in w2v_pretrained]
            vec_avg = sum(vecs) / len(vecs) if vecs else [0] * 300
            vectors.append(vec_avg)
        vectors = pd.DataFrame(vectors)
        vectors.columns = vectors.columns.astype(str)
    else:
        raise ValueError("Invalid vectorization technique. Choose from 'BoW', 'TFIDF', or 'Word2Vec'.")
    return pd.DataFrame(vectors)

vect_tech = 'BoW'

X = getTrainSet(df, vect_tech)

y_se = df[y_column].map(lambda x: 1 if x == 'SE' else 0)
y_us = df[y_column].map(lambda x: 1 if x == 'US' else 0)
y_o = df[y_column].map(lambda x: 1 if x == 'O' else 0)
y_pe = df[y_column].map(lambda x: 1 if x == 'PE' else 0)

# def getTrainSetBoW(df):
#     countvec = CountVectorizer(max_features=100)
#     bow = countvec.fit_transform(df[X_column]).toarray()
#     training_data = pd.DataFrame(bow)
#     training_data.columns = training_data.columns.astype(str)
#     return training_data

# def getTrainSetTFIDF(df):
#     countvec = CountVectorizer(max_features=100)
#     bow = countvec.fit_transform(df[X_column]).toarray()
#     tfidfconverter = TfidfTransformer()
#     X = tfidfconverter.fit_transform(bow).toarray()
#     training_data = pd.DataFrame(X)
#     training_data.columns = training_data.columns.astype(str)
#     return training_data

# def getTrainSetWord2Vec(df):
#     w2v_pretrained = gensim.downloader.load('word2vec-google-news-300')
#     training_data = []
#     for msg in df[X_column]:
#         words = msg.split()
#         vecs = []
#         for word in words:
#             if word in w2v_pretrained:
#                 vecs.append(w2v_pretrained[word][:300])
#         if vecs:
#             vec_avg = sum(vecs) / len(vecs)
#         else:
#             vec_avg = [0] * 300
#         training_data.append(vec_avg)

#     training_data = pd.DataFrame(training_data)
#     training_data.columns = training_data.columns.astype(str)
    # return training_data

# Training

In [32]:
def custom_metrics(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return {"precision": precision, "recall": recall}

In [33]:
global_results = []

for y in [y_se, y_us, y_o, y_pe]:
    results = pd.DataFrame(columns=["Fold", "Model", "Accuracy", "F1-Score"])
    fold = KFold(n_splits=10, random_state=SEED, shuffle=True)
    foldcounter = 1
    for train_index, test_index in fold.split(X, y):
        print("Processing Fold "+ str(foldcounter) + " ...")
        X_train, X_test, y_train, y_test = \
        X[X.index.isin(train_index)], X[X.index.isin(test_index)], y[train_index], y[test_index]
        clf = LazyClassifier(verbose=-100, ignore_warnings=True, custom_metric=custom_metrics)
        models, predictions = clf.fit(X_train, X_test, y_train, y_test)
        for model in models[:].iterrows():
            result = pd.DataFrame({
                "Fold": [foldcounter],
                "Model": [model[0]],
                "Accuracy": [round(model[1][0], 3)],
                "F1-Score": [round(model[1][3], 3)]})
            results = results.append(result)
        foldcounter += 1
    global_results.append(results)

Processing Fold 1 ...


100%|██████████| 29/29 [00:01<00:00, 15.94it/s]


[LightGBM] [Info] Number of positive: 113, number of negative: 205
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 70
[LightGBM] [Info] Number of data points in the train set: 318, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.355346 -> initscore=-0.595622
[LightGBM] [Info] Start training from score -0.595622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


100%|██████████| 29/29 [00:01<00:00, 15.22it/s]


[LightGBM] [Info] Number of positive: 113, number of negative: 205
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000167 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 68
[LightGBM] [Info] Number of data points in the train set: 318, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.355346 -> initscore=-0.595622
[LightGBM] [Info] Start training from score -0.595622
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

[LightGBM] [Info] Number of positive: 60, number of negative: 259
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 70
[LightGBM] [Info] Number of data points in the train set: 319, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.188088 -> initscore=-1.462483
[LightGBM] [Info] Start training from score -1.462483
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

100%|██████████| 29/29 [00:01<00:00, 17.32it/s]


[LightGBM] [Info] Number of positive: 56, number of negative: 263
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 319, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.175549 -> initscore=-1.546802
[LightGBM] [Info] Start training from score -1.546802
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

100%|██████████| 29/29 [00:01<00:00, 16.80it/s]

[LightGBM] [Info] Number of positive: 57, number of negative: 262
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000157 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 68
[LightGBM] [Info] Number of data points in the train set: 319, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.178683 -> initscore=-1.525293
[LightGBM] [Info] Start training from score -1.525293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [34]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,custom_metrics,Time Taken
Model,,,,,,
NearestCentroid,0.97,0.98,0.98,0.97,"{'precision': 0.9, 'recall': 1.0}",0.03
BernoulliNB,0.94,0.93,0.93,0.94,"{'precision': 0.8888888888888888, 'recall': 0....",0.02
ExtraTreeClassifier,0.89,0.89,0.89,0.89,"{'precision': 0.7272727272727273, 'recall': 0....",0.02
RandomForestClassifier,0.91,0.87,0.87,0.91,"{'precision': 0.875, 'recall': 0.7777777777777...",0.24
BaggingClassifier,0.86,0.87,0.87,0.86,"{'precision': 0.6666666666666666, 'recall': 0....",0.06
DecisionTreeClassifier,0.86,0.87,0.87,0.86,"{'precision': 0.6666666666666666, 'recall': 0....",0.02
LGBMClassifier,0.91,0.83,0.83,0.91,"{'precision': 1.0, 'recall': 0.6666666666666666}",0.09
XGBClassifier,0.91,0.83,0.83,0.91,"{'precision': 1.0, 'recall': 0.6666666666666666}",0.06
ExtraTreesClassifier,0.83,0.81,0.81,0.83,"{'precision': 0.6363636363636364, 'recall': 0....",0.16


# Security requirements

In [35]:
grouped = global_results[0].groupby("Model")
print(grouped[["Accuracy", "F1-Score"]].mean().sort_values(by="Accuracy", ascending=False), "\n\n")

# for key, item in grouped:
#     print(grouped.get_group(key), "\n\n")

                               Accuracy  F1-Score
Model                                            
BaggingClassifier                  0.87      0.87
BernoulliNB                        0.87      0.87
AdaBoostClassifier                 0.87      0.87
ExtraTreesClassifier               0.87      0.87
RandomForestClassifier             0.86      0.86
NearestCentroid                    0.86      0.86
XGBClassifier                      0.86      0.85
SGDClassifier                      0.85      0.85
CategoricalNB                      0.85      0.84
LogisticRegression                 0.84      0.85
NuSVC                              0.84      0.84
SVC                                0.84      0.84
Perceptron                         0.84      0.84
RidgeClassifier                    0.84      0.83
LinearDiscriminantAnalysis         0.84      0.83
RidgeClassifierCV                  0.83      0.83
PassiveAggressiveClassifier        0.83      0.83
CalibratedClassifierCV             0.82      0.81


# Usability requirements

In [36]:
grouped = global_results[1].groupby("Model")
print(grouped[["Accuracy", "F1-Score"]].mean().sort_values(by="Accuracy", ascending=False), "\n\n")

# for key, item in grouped:
#     print(grouped.get_group(key), "\n\n")

                               Accuracy  F1-Score
Model                                            
NuSVC                              0.86      0.79
RandomForestClassifier             0.85      0.84
RidgeClassifierCV                  0.84      0.83
SVC                                0.83      0.80
KNeighborsClassifier               0.83      0.82
NearestCentroid                    0.83      0.83
BernoulliNB                        0.83      0.82
XGBClassifier                      0.83      0.82
ExtraTreesClassifier               0.83      0.82
LinearDiscriminantAnalysis         0.82      0.82
LogisticRegression                 0.82      0.82
RidgeClassifier                    0.82      0.82
AdaBoostClassifier                 0.81      0.81
SGDClassifier                      0.80      0.80
CategoricalNB                      0.80      0.74
BaggingClassifier                  0.80      0.79
Perceptron                         0.79      0.80
LGBMClassifier                     0.79      0.76


# Operational requirements

In [37]:
grouped = global_results[2].groupby("Model")
print(grouped[["Accuracy", "F1-Score"]].mean().sort_values(by="Accuracy", ascending=False), "\n\n")

# for key, item in grouped:
#     print(grouped.get_group(key), "\n\n")

                               Accuracy  F1-Score
Model                                            
BernoulliNB                        0.87      0.86
RidgeClassifier                    0.86      0.85
LogisticRegression                 0.86      0.86
LinearDiscriminantAnalysis         0.86      0.85
NearestCentroid                    0.86      0.86
SVC                                0.86      0.83
RidgeClassifierCV                  0.86      0.85
SGDClassifier                      0.85      0.84
Perceptron                         0.85      0.85
AdaBoostClassifier                 0.84      0.83
XGBClassifier                      0.84      0.82
RandomForestClassifier             0.84      0.83
CategoricalNB                      0.83      0.80
CalibratedClassifierCV             0.83      0.80
QuadraticDiscriminantAnalysis      0.83      0.83
LabelPropagation                   0.83      0.81
LabelSpreading                     0.83      0.81
PassiveAggressiveClassifier        0.83      0.83


# Performance requirements

In [38]:
grouped = global_results[3].groupby("Model")
print(grouped[["Accuracy", "F1-Score"]].mean().sort_values(by="Accuracy", ascending=False), "\n\n")

# for key, item in grouped:
#     print(grouped.get_group(key), "\n\n")

                               Accuracy  F1-Score
Model                                            
NearestCentroid                    0.92      0.92
BernoulliNB                        0.92      0.91
RidgeClassifierCV                  0.90      0.89
RidgeClassifier                    0.90      0.89
XGBClassifier                      0.90      0.89
LogisticRegression                 0.90      0.89
LGBMClassifier                     0.89      0.88
RandomForestClassifier             0.89      0.89
ExtraTreesClassifier               0.89      0.89
LinearDiscriminantAnalysis         0.89      0.88
SGDClassifier                      0.88      0.88
AdaBoostClassifier                 0.88      0.88
SVC                                0.88      0.86
QuadraticDiscriminantAnalysis      0.87      0.84
KNeighborsClassifier               0.87      0.85
CalibratedClassifierCV             0.87      0.84
CategoricalNB                      0.86      0.82
ExtraTreeClassifier                0.85      0.85
